## Analysis of Game of Thrones Co-Occurrence Network

<p>This notebook aims to explore the co-occurrence network of characters in the popular book series "A Song of Ice and Fire" by George R.R. Martin, which was adapted into the equally popular HBO television series "Game of Thrones." </p>

<p>
The co-occurrence of characters is defined by their names appearing within a 15-word distance in the books.
</p>
<p>
Through this analysis, we aim to gain insights into the relationships and interactions between the characters in the series.
</p>

<p><img src="https://assets.datacamp.com/production/project_76/img/got_network.jpeg" style="width: 550px"></p>

## 1. Loading the Dataset

<p>This dataset constitutes a network and is given as a text file describing the <em>edges</em> between characters, with some attributes attached to each edge. Let's start by loading in the data for the first book <em>A Game of Thrones</em> and inspect it.</p>

In [39]:
# loading datasets
!wget -qnc -P datasets https://raw.githubusercontent.com/Ozzey/GOT_Network_Analysis/main/dataset/book1.csv
!wget -qnc -P datasets https://raw.githubusercontent.com/Ozzey/GOT_Network_Analysis/main/dataset/book2.csv
!wget -qnc -P datasets https://raw.githubusercontent.com/Ozzey/GOT_Network_Analysis/main/dataset/book3.csv
!wget -qnc -P datasets https://raw.githubusercontent.com/Ozzey/GOT_Network_Analysis/main/dataset/book4.csv
!wget -qnc -P datasets https://raw.githubusercontent.com/Ozzey/GOT_Network_Analysis/main/dataset/book5.csv

# Importing modules
import pandas as pd

# Reading in datasets/book1.csv
book1 = pd.read_csv('datasets/book1.csv')
book2 = pd.read_csv('datasets/book2.csv')
book3 = pd.read_csv('datasets/book3.csv')
book4 = pd.read_csv('datasets/book4.csv')
book5 = pd.read_csv('datasets/book5.csv')

# Printing out the head of the dataset
book3.head()

,Source,Target,Type,weight,book
0,Addam-Marbrand,Gyles-Rosby,Undirected,3,3
1,Addam-Marbrand,Jaime-Lannister,Undirected,3,3
2,Addam-Marbrand,Jalabhar-Xho,Undirected,3,3
3,Addam-Marbrand,Joffrey-Baratheon,Undirected,3,3
4,Addam-Marbrand,Kevan-Lannister,Undirected,3,3


In [40]:
# Importing modules
import networkx as nx

# Creating an empty graph object
G_book1 = nx.Graph()

# Iterating through the DataFrame to add edges
for _, edge in book1.iterrows():
    G_book1.add_edge(edge['Source'], edge['Target'], weight=edge['weight'])

# Creating a list of networks for all the books
books = [G_book1]
book_fnames = ['datasets/book2.csv', 'datasets/book3.csv', 'datasets/book4.csv', 'datasets/book5.csv']
for book_fname in book_fnames:
    book = pd.read_csv(book_fname)
    G_book = nx.Graph()
    for _, edge in book.iterrows():
        G_book.add_edge(edge['Source'], edge['Target'], weight=edge['weight'])
    books.append(G_book)

In [41]:
import pandas as pd
import networkx as nx
import random
import plotly.graph_objects as go

# Set the color palettes
node_colors = ['#4c0033', '#0c134f', '#917fb3', '#aa77ff']
edge_colors = ['#2D033B', '#fc2947', '#62cdff', '#ffb84c']

# Load the dataset
df = pd.read_csv('datasets/book1.csv')

# Create a networkx graph
G = nx.from_pandas_edgelist(df, source='Source', target='Target')

# Remove 50% of the edges
edges = list(G.edges())
edges_to_remove = random.sample(edges, k=len(edges)//2)
G.remove_edges_from(edges_to_remove)

# Calculate node positions
pos = nx.spring_layout(G, k=0.5)

# Get the degree centrality for each node
degree_centrality = nx.degree_centrality(G)

# Create a list of edge trace objects
edge_traces = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace = go.Scatter(x=[x0, x1, None], y=[y0, y1, None], mode='lines', line=dict(color=random.choice(edge_colors)), hoverinfo='none')
    edge_traces.append(edge_trace)

# Create a list of node trace objects
node_traces = []
for i in range(len(G.nodes())):
    node_size = 10 + degree_centrality[list(G.nodes())[i]] * 120
    if node_size > 15:
        x, y = pos[list(G.nodes())[i]]
        node_trace = go.Scatter(x=[x], y=[y], mode='markers+text', text=[list(G.nodes())[i]], textposition='bottom center', textfont=dict(color='white', size=14, family='Arial, sans-serif'), marker=dict(size=node_size, color=random.choice(node_colors)), hovertemplate='%{text}<extra></extra>', hoverinfo='text')
        node_traces.append(node_trace)
    else:
        x, y = pos[list(G.nodes())[i]]
        node_trace = go.Scatter(x=[x], y=[y], mode='markers', marker=dict(size=node_size, color=random.choice(node_colors)), hovertemplate='', hoverinfo='none')
        node_traces.append(node_trace)

# Create the plotly figure
fig = go.Figure(data=edge_traces + node_traces, layout=go.Layout(
    title='Game of Thrones Social Network',
    template='plotly_dark',
    margin=dict(b=20, l=5, r=5, t=40),
    hovermode='closest',
    xaxis=dict(showgrid=True, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=True, zeroline=False, showticklabels=False),
    width=1200,
    height=800,
    plot_bgcolor='#181818', # Set the canvas background color to transparent
    paper_bgcolor='rgba(0,0,0,0)' # Set the paper background color to transparent
))

# Remove legends
fig.update_layout(showlegend=False)

# Show the figure
fig.show()

---
# **1. DESCRIPTION OF THE NETWORK**
---

## **Creating the Network of Thrones**

The DataFrame `book1` contains five columns, namely `Source`, `Target`, `Type`, `weight`, and `book`. The `Source` and `Target` columns represent the two nodes connected by an edge, while the `Type` column specifies whether the edge is directed or undirected. In this case, all edges are undirected. The `weight` attribute of each edge indicates the number of interactions between the characters in the book, and the `book` column indicates the book number.

To create a network from this data, we will use the `networkx` library, which is commonly used for network analysis. Specifically, we will create a graph object for the first book in the series.

### Calculating Number of Nodes and Edges

In [42]:
import pandas as pd

def node_edges():
  # Define the list of CSV files
  csv_files = ['datasets/book1.csv', 'datasets/book2.csv', 'datasets/book3.csv', 'datasets/book4.csv', 'datasets/book5.csv']

  # Define empty lists to store all nodes and edges
  all_nodes = []
  all_edges = []

  # Loop through all CSV files and extract nodes and edges
  for csv_file in csv_files:
      # Load the CSV file into a pandas DataFrame
      df = pd.read_csv(csv_file)
      # Extract all unique nodes (characters)
      nodes = pd.concat([df['Source'], df['Target']]).unique().tolist()
      # Add the nodes to the list of all nodes
      all_nodes.extend(nodes)
      # Extract the edges and add them to the list of all edges
      edges = list(zip(df['Source'], df['Target']))
      all_edges.extend(edges)

  # Count the number of unique nodes and edges
  num_nodes = len(set(all_nodes))
  num_edges = len(set(all_edges))

  return (num_nodes,num_edges)

num_nodes = node_edges()[0]
num_edges = node_edges()[1]

# Print the results
print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)


Number of nodes: 796
Number of edges: 2823


### **Network Statistics**

In [43]:
import networkx as nx
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio

pio.templates.default = "plotly_dark"

# Load the co-occurrence networks for all the books
books = []
book_fnames = ['datasets/book1.csv', 'datasets/book2.csv', 'datasets/book3.csv',
               'datasets/book4.csv', 'datasets/book5.csv']
for book_fname in book_fnames:
    book = pd.read_csv(book_fname)
    G_book = nx.Graph()
    for _, edge in book.iterrows():
        G_book.add_edge(edge['Source'], edge['Target'], weight=edge['weight'])
    books.append(G_book)

# Calculate the number of nodes and edges in the network
num_nodes = node_edges()[0]
num_edges = node_edges()[1]

# Calculate the diameter and radius of the network
diameter = nx.diameter(books[0])
radius = nx.radius(books[0])

# Calculate the global and average local clustering coefficients
global_cc = nx.transitivity(books[0])
local_cc = nx.average_clustering(books[0])

# Calculate the shortest path lengths between all pairs of nodes
all_shortest_paths = dict(nx.shortest_path_length(books[0]))

# Extract the shortest path lengths as a list of values
shortest_path_lengths = []
for source in all_shortest_paths:
    for target in all_shortest_paths[source]:
        shortest_path_lengths.append(all_shortest_paths[source][target])

# Calculate the average path length
avg_path_length = sum(shortest_path_lengths) / len(shortest_path_lengths)

# Create the global and local clustering coefficient plot
cc_fig = go.Figure()
cc_fig.add_trace(go.Table(
    header=dict(values=['Network Statistic', 'Value'],
                fill_color='#636efa',
                align='left'),
    cells=dict(values=[['Number of Nodes', 'Number of Edges', 'Diameter', 'Radius', 'Global Clustering Coefficient', 'Average Local Clustering Coefficient', 'Average Path Length'],
                       [num_nodes, num_edges, diameter, radius, f'{global_cc:.2f}', f'{local_cc:.2f}', f'{avg_path_length:.2f}']],
               align='left')
))
cc_fig.add_trace(go.Histogram(
    x=shortest_path_lengths,
    nbinsx=20,
    marker_color='#636efa',
    opacity=0.75,
    name='Shortest Path Lengths',
    xaxis='x2',
    yaxis='y2',
))
cc_fig.update_layout(
    title={'text': "Network Statistics"},
    grid={'rows': 1, 'columns': 2},
    xaxis=dict(domain=[0, 0.45]),
    xaxis2=dict(domain=[0.55, 1]),
    yaxis2=dict(anchor='x2'),
    height=400,
    width=1000,
    showlegend=False,
)

# Create the average path length histogram
pl_fig = go.Figure()
pl_fig.add_trace(go.Histogram(
    x=shortest_path_lengths,
    nbinsx=20,
    marker_color='#636efa',
    opacity=0.75,
    name='Shortest Path Lengths',
))
pl_fig.update_layout(
    title={'text': "Histogram of Shortest Path Lengths"},
    xaxis_title="Shortest Path Length",
    yaxis_title="Count",
    height=400,
    width=1000,
)

# Show the plots
cc_fig.show()
pl_fig.show()

# **2. DEGREE DISTRIBUTION**

## **The distribution algorithm**
It is the spring layout distribution algorithm, which is a force-directed layout algorithm that simulates the forces between nodes in the graph to determine their positions. 

In [44]:
import networkx as nx
import pandas as pd
import numpy as np
import random
import plotly.graph_objs as go
import plotly.io as pio

pio.templates.default = "plotly_dark"

# Load the co-occurrence network
book_df = pd.read_csv('datasets/book1.csv')

# Create a weighted graph from the edge list
G = nx.from_pandas_edgelist(book_df, source='Source', target='Target', edge_attr='weight')

# Calculate betweenness centrality for all nodes
betweenness_centrality = nx.betweenness_centrality(G, weight='weight')

# Choose the node with highest betweenness centrality as the hub
hub_node = max(betweenness_centrality, key=betweenness_centrality.get)

# Create a hub graph by connecting the hub node to a random subset of the other nodes
hub_graph = nx.Graph()
hub_graph.add_node(hub_node)
hub_neighbors = list(G.neighbors(hub_node))
for neighbor in hub_neighbors:
    hub_graph.add_edge(hub_node, neighbor)
    for sub_neighbor in G.neighbors(neighbor):
        if sub_neighbor != hub_node and random.random() < 0.5:
            hub_graph.add_edge(neighbor, sub_neighbor)

# Create a plot of the hub graph
hub_fig = go.Figure()
pos = nx.spring_layout(hub_graph)
for edge in hub_graph.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_color = hub_fig.data[-1].marker.color if len(hub_fig.data) > 0 and hub_fig.data[-1].x[0] == x0 and hub_fig.data[-1].y[0] == y0 else 'white'
    hub_fig.add_trace(go.Scatter(x=[x0, x1, None], y=[y0, y1, None], mode='lines', line=dict(color=edge_color, width=1), hoverinfo='none', showlegend=False))
for node in hub_graph.nodes():
    x, y = pos[node]
    size = 75 * betweenness_centrality[node]
    color = f'rgb({random.randint(0, 255)}, {random.randint(0, 255)}, {random.randint(0, 255)})'
    label = node if node != hub_node else ''
    hub_fig.add_trace(go.Scatter(x=[x], y=[y], mode='markers', marker=dict(size=size, color=color), text=label, hovertemplate="", showlegend=False))
hub_fig.update_layout(title={'text': 'Hub Graph for Book1 Dataset', 'x': 0.5}, height=800, width=1200, xaxis_title='Distribution', yaxis_title='Weight')

# Show the plot
hub_fig.show()

The hub graph provides a useful visualization tool for analyzing the co-occurrence network in the book. It allows us to identify the most important characters and their relationships, and the use of the spring layout algorithm ensures that the graph is easy to interpret.

## **MLE Model**: Understanding relationship between characters

we can use the **MLE graph** to compare the degree of similarity between named entities across the five books. If the distribution of edge weights is similar across the books, then it suggests that the relationships between named entities are consistent across the series. On the other hand, if the distribution of edge weights varies significantly across the books, then it suggests that the relationships between named entities are different in each book.

In [45]:
import pandas as pd
import numpy as np
from scipy.stats import norm

def MLE():

  # Load the data from CSV files into a Pandas DataFrame
  data = pd.concat([pd.read_csv(f) for f in ['datasets/book1.csv', 'datasets/book2.csv', 'datasets/book3.csv', 'datasets/book4.csv', 'datasets/book5.csv']])

  # Compute the mean and standard deviation of the edge weights
  mu = np.mean(data['weight'])
  sigma = np.std(data['weight'])

  # Define the likelihood function for the Gaussian distribution
  def likelihood(x):
      return norm.pdf(x, mu, sigma)

  # Compute the log-likelihood of the data
  log_likelihood = np.sum(np.log(likelihood(data['weight'])))

  # Print the estimated parameters and log-likelihood
  print('Mean:', mu)
  print('Standard deviation:', sigma)
  print('Log-likelihood:', log_likelihood)

  return (mu,sigma,log_likelihood)

In [46]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio
from plotly.subplots import make_subplots

pio.templates.default = "plotly_dark"


# Get the MLE estimates for the mean and standard deviation
mu, sigma, log_likelihood = MLE()

# Round the estimated parameters to two decimal places
mu_rounded = round(mu, 2)
sigma_rounded = round(sigma, 2)
log_likelihood_rounded = round(log_likelihood, 2)

# Define the likelihood function for a Gaussian distribution
def likelihood(x, mu, sigma):
    return np.exp(-(x - mu)**2 / (2 * sigma**2)) / (np.sqrt(2 * np.pi) * sigma)

# Create a histogram of the edge weights
data = pd.concat([pd.read_csv(f) for f in ['datasets/book1.csv', 'datasets/book2.csv', 'datasets/book3.csv', 'datasets/book4.csv', 'datasets/book5.csv']])
histogram = go.Histogram(
    x=data['weight'],
    histnorm='probability density',
    name='Histogram of Edge Weights',
    marker=dict(color='#636EFA', line=dict(color='white', width=1))
)

# Create a trace for the estimated Gaussian distribution
x = np.linspace(mu - 3*sigma, mu + 3*sigma, 1000)
gaussian = go.Scatter(
    x=x,
    y=likelihood(x, mu, sigma),
    mode='lines',
    name='Estimated Gaussian Distribution'
)

# Create a table of the estimated parameters
table = go.Table(
    header=dict(values=['Parameter', 'Value']),
    cells=dict(values=[['Mean', 'Standard Deviation', 'Log-Likelihood'],
                        [mu_rounded, sigma_rounded, log_likelihood_rounded]])
)

# Create subplots for the histogram and Gaussian distribution, and the table
fig = make_subplots(rows=1, cols=2, column_widths=[0.7, 0.3], specs=[[{"type": "xy"}, {"type": "table"}]])

# Add the traces to the subplots
fig.add_trace(histogram, row=1, col=1)
fig.add_trace(gaussian, row=1, col=1)
fig.add_trace(table, row=1, col=2)

# Set the layout of the figure
fig.update_layout(
    title={'text': "Histogram and Estimated Gaussian Distribution of Edge Weights", 'x':0.5},
    height=400,
    width=1000,
    showlegend=False,
    xaxis_title='Edge Weights',
    yaxis_title='Probability Density',
    xaxis=dict(range=[0, mu + 3*sigma]),
    bargap=0.05,
    margin=dict(l=50, r=50, t=50, b=50),
)

# Show the plot
fig.show()

Mean: 8.347147608083908
Standard deviation: 11.146411843291025
Log-likelihood: -14971.689574561498


## **Comparison with Other Graphs**

In [47]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio
import networkx as nx

pio.templates.default = "plotly_dark"

# Load the edge list data
data = pd.concat([pd.read_csv(f) for f in ['datasets/book1.csv', 'datasets/book2.csv', 'datasets/book3.csv', 'datasets/book4.csv', 'datasets/book5.csv']])

# Define a function to compute the network statistics
def compute_network_statistics(data):
    # Create a graph from the edge list
    G = nx.from_pandas_edgelist(data, 'Source', 'Target', ['weight'])

    # Compute the number of nodes and edges
    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()

    # Compute the degree distribution
    degree_sequence = sorted([d for n, d in G.degree()], reverse=True)
    degree_counts = np.unique(degree_sequence, return_counts=True)

    # Compute the clustering coefficient
    clustering_coefficient = nx.average_clustering(G)

    # Compute the average shortest path length
    avg_shortest_path_length = nx.average_shortest_path_length(G)

    # Compute the expected number of edges in the Erdős-Rényi model
    p = 2*num_edges / (num_nodes*(num_nodes-1))
    expected_edges = int(p * num_nodes * (num_nodes-1) / 2)

    # Generate an Erdős-Rényi random graph with the same number of nodes and expected number of edges
    ER = nx.erdos_renyi_graph(num_nodes, p)

    # Compute the degree distribution, clustering coefficient, and average path length of the Erdős-Rényi model
    ER_degree_sequence = sorted([d for n, d in ER.degree()], reverse=True)
    ER_degree_counts = np.unique(ER_degree_sequence, return_counts=True)
    ER_clustering_coefficient = nx.average_clustering(ER)

    # Return the network statistics as a dictionary
    return {
        'Network': {'Number of nodes': num_nodes, 'Number of edges': num_edges, 'Average degree': np.mean(degree_sequence), 'Clustering coefficient': clustering_coefficient},
        'Erdős-Rényi model': {'Number of nodes': num_nodes, 'Number of edges': expected_edges, 'Average degree': np.mean(ER_degree_sequence), 'Clustering coefficient': ER_clustering_coefficient}
    }

# Compute the network statistics
stats = compute_network_statistics(data)

# Define the table of parameters
table = go.Table(
    header=dict(values=['Model', 'Number of nodes', 'Number of edges', 'Average degree', 'Clustering coefficient']),
    cells=dict(values=[['Network', 'Erdős-Rényi model'], 
                        [stats['Network']['Number of nodes'], stats['Erdős-Rényi model']['Number of nodes']], 
                        [stats['Network']['Number of edges'], stats['Erdős-Rényi model']['Number of edges']], 
                        [f"{stats['Network']['Average degree']:.15f}", f"{stats['Erdős-Rényi model']['Average degree']:.15f}"], 
                        [f"{stats['Network']['Clustering coefficient']:.15f}", f"{stats['Erdős-Rényi model']['Clustering coefficient']:.15f}"]],
    # Change background color and font color of the table
    fill_color=['#6A359C', '#804FB3'],  # Set different colors for each row
    font=dict(color='white')
)
)
# Create a figure with only the table
fig = go.Figure(data=[table])

# Set the layout of the figure
fig.update_layout(
    title='Erdős-Rényi Random Graph',
    width=800,
    height=150,
    margin=dict(l=20, r=20, t=60, b=20),
    hovermode='closest',
    paper_bgcolor = "rgba(0,0,0,0)"
)

# Show the plot
fig.show()

Based on the dat, we can see that our network has a higher clustering coefficient (0.486) compared to the Erdős-Rényi model (0.007). This indicates that our network has a higher tendency for nodes to form clusters or groups, which is a characteristic commonly found in real-world networks.

Additionally, the average degree of our network (7.093) is slightly higher than that of the Erdős-Rényi model (7.040), which means that our network has more edges on average than a random network with the same number of nodes and edges.

Overall, the clustering coefficient and the average degree of our network indicate that it has a more complex structure than a random network with the same number of nodes and edges.



---
# **3. DEGREE CENTRALITIES**
---




## **Degree Centrality:** Finding the Most Important Character in Game of Thrones
In network science, there are many different metrics to measure the importance of a node in a network. One such measure is degree centrality, which looks at the number of neighbors a node has, i.e., the number of nodes it is connected to. A node with a high degree centrality is considered important in the network.

Using this measure, we can extract the top 10 important characters from the first and fifth books of the Game of Thrones series. Here's some example code to do this:

In [48]:
import plotly.graph_objs as go

# Calculating the degree centrality of book 1
deg_cen_book1 = nx.degree_centrality(books[1])

# Calculating the degree centrality of book 5
deg_cen_book5 = nx.degree_centrality(books[2])

# Sorting the dictionaries according to their degree centrality and extracting the top 10
top10_book1 = sorted(deg_cen_book1.items(), key=lambda x:x[1], reverse=True)[0:10]

# Sorting the dictionaries according to their degree centrality and extracting the top 10
top10_book5 = sorted(deg_cen_book5.items(), key=lambda x:x[1], reverse=True)[0:10]

# Creating a bar chart of the top 10 characters by degree centrality in each book
fig = go.Figure(data=[go.Bar(x=[char[0] for char in top10_book1],
                             y=[char[1] for char in top10_book1],
                             name='Book 1',
                             marker_color='#6A359C'),
                      go.Bar(x=[char[0] for char in top10_book5],
                             y=[char[1] for char in top10_book5],
                             name='Book 5',
                             marker_color='#820021')])

# Updating the layout
fig.update_layout(title="Top 10 Characters by Degree Centrality in Book 1 and Book 5",
                  xaxis_title="Character",
                  yaxis_title="Degree Centrality",
                  barmode='group',
                  template='plotly_dark')

# Updating the font size of the tick labels
fig.update_xaxes(tickfont=dict(size=12))
fig.update_yaxes(tickfont=dict(size=12))

# Displaying the figure
fig.show()

## **Analyzing the Evolution of Character Importance**

The importance of characters in the **Game of Thrones** series changes over the course of the five books. While **Eddard Stark** is the most important character in the first book according to degree centrality, he doesn't even appear in the top 10 of the fifth book. This is because the importance of characters can shift as the story progresses and events unfold.

To examine this evolution of character importance, we can focus on a few key characters who were among the top 10 in degree centrality in the first book, such as **Eddard Stark , Jon Snow**, and **Tyrion**.

Here's an example code block that calculates the degree centrality of these characters in each book and displays the results:

In [49]:
import plotly.graph_objs as go

# Creating a list of degree centrality of all the books
evol = [nx.degree_centrality(book) for book in books]

# Creating a DataFrame from the list of degree centralities in all the books
degree_evol_df = pd.DataFrame.from_records(evol)

# Creating traces for Eddard-Stark, Tyrion-Lannister, and Jon-Snow
fig = go.Figure()
fig.add_trace(go.Scatter(x=degree_evol_df.index, y=degree_evol_df['Eddard-Stark'], mode='lines', name='Eddard-Stark'))
fig.add_trace(go.Scatter(x=degree_evol_df.index, y=degree_evol_df['Tyrion-Lannister'], mode='lines', name='Tyrion-Lannister'))
fig.add_trace(go.Scatter(x=degree_evol_df.index, y=degree_evol_df['Jon-Snow'], mode='lines', name='Jon-Snow'))

# Updating the layout
fig.update_layout(title="Evolution of Character Importance According to Degree Centrality",
                  xaxis_title="Book",
                  yaxis_title="Degree Centrality",
                  legend=dict(font=dict(size=12)),
                  template='plotly_dark')

# Updating the font size of the tick labels
fig.update_xaxes(tickfont=dict(size=12))
fig.update_yaxes(tickfont=dict(size=12))

# Displaying the figure
fig.show()

## **Closeness Centrality:** Analyzing Stannis Baratheon and Other Important Characters

To gain a deeper understanding of the Game of Thrones character co-occurrence network, we can analyze various measures such as betweenness centrality and PageRank. In this case, we will focus on betweenness centrality and plot the evolution of the top four characters in each book.

This analysis will help us identify important characters in the network and uncover any interesting trends or patterns. Specifically, we are interested in understanding the role of Stannis Baratheon in the network and how his importance changes over the course of the five books.

To do this, we will plot the evolution of betweenness centrality for Stannis Baratheon and compare it to other important characters in the network. This will allow us to see how Stannis' importance changes in relation to other characters and whether he plays a significant role in the network overall.

In [50]:
import plotly.graph_objs as go

# Creating a list of betweenness centrality of all the books just like we did for degree centrality
evol = [nx.betweenness_centrality(book, weight='weight') for book in books]

# Making a DataFrame from the list
betweenness_evol_df = pd.DataFrame.from_records(evol).fillna(0)

# Finding the top 4 characters in every book
set_of_char = set()
for i in range(5):
    set_of_char |= set(list(betweenness_evol_df.T[i].sort_values(ascending=False)[0:4].index))
list_of_char = list(set_of_char)

# Creating traces for each character
fig = go.Figure()
for char in list_of_char:
    fig.add_trace(go.Scatter(x=betweenness_evol_df.index, y=betweenness_evol_df[char], mode='lines', name=char))

# Updating the layout
fig.update_layout(title="Evolution of Important Characters According to Betweenness Centrality",
                  xaxis_title="Book",
                  yaxis_title="Betweenness Centrality",
                  legend=dict(font=dict(size=12)),
                  template='plotly_dark')

# Updating the font size of the tick labels
fig.update_xaxes(tickfont=dict(size=12))
fig.update_yaxes(tickfont=dict(size=12))

# Displaying the figure
fig.show()

## **Page-Rank:** Analyzing Google PageRank in the Game of Thrones Network

In addition to <strong>betweenness centrality</strong>, we can also analyze the importance of characters in the Game of Thrones network according to <strong>Google PageRank</strong>. PageRank is a measure of the importance of a webpage in the world wide web, based on the analysis of its inlinks and outlinks.

In the case of the Game of Thrones network, we can use PageRank to gain a deeper understanding of the importance of characters in the network and uncover any interesting trends or patterns. Specifically, we are interested in understanding the role of <strong>Stannis Baratheon</strong> in the network and how his importance changes over the course of the five books, according to PageRank.

To do this, we will calculate the PageRank of each character in the network and compare it to their <strong>degree centrality</strong> and betweenness centrality. This will allow us to see how Stannis' importance changes in relation to other characters and whether he plays a significant role in the network overall.

In [51]:
import plotly.graph_objs as go

# Creating a list of pagerank of all the characters in all the books
evol = [nx.pagerank(book) for book in books]

# Making a DataFrame from the list
pagerank_evol_df = pd.DataFrame.from_records(evol).fillna(0)

# Finding the top 4 characters in every book
set_of_char = set()
for i in range(5):
    set_of_char |= set(list(pagerank_evol_df.T[i].sort_values(ascending=False)[0:4].index))
list_of_char = list(set_of_char)

# Creating traces for each character
fig = go.Figure()
for char in list_of_char:
    fig.add_trace(go.Scatter(x=pagerank_evol_df.index, y=pagerank_evol_df[char], mode='lines', name=char))

# Updating the layout
fig.update_layout(title="Evolution of Important Characters According to PageRank",
                  xaxis_title="Book",
                  yaxis_title="PageRank",
                  legend=dict(font=dict(size=12)),
                  width=1200, height=600,
                  template='plotly_dark')

# Updating the font size of the tick labels
fig.update_xaxes(tickfont=dict(size=12))
fig.update_yaxes(tickfont=dict(size=12))

# Displaying the figure
fig.show()

## **KATZ CENTRALITY**: Influence of characters according to Katz Centrality

Katz centrality is a measure of node importance in a network that takes into account both the direct and indirect influence of a node's neighbors.

In [52]:
import plotly.graph_objs as go

# Creating a list of katzcentrality of all the characters in all the books
evol = [nx.katz_centrality_numpy(book) for book in books]

# Making a DataFrame from the list
katzcentrality_evol_df = pd.DataFrame.from_records(evol).fillna(0)

# Finding the top 4 characters in every book
set_of_char = set()
for i in range(5):
    set_of_char |= set(list(katzcentrality_evol_df.T[i].sort_values(ascending=False)[0:4].index))
list_of_char = list(set_of_char)

# Creating traces for each character
fig = go.Figure()
for char in list_of_char:
    fig.add_trace(go.Scatter(x=katzcentrality_evol_df.index + 1, y=katzcentrality_evol_df[char], mode='lines', name=char))

# Updating the layout
fig.update_layout(title="Influence of characters according to Katz Centrality",
                  xaxis_title="Book",
                  yaxis_title="Katz Centrality",
                  legend=dict(font=dict(size=12)),
                  width=1200, height=600,
                  template='plotly_dark')

# Updating the font size of the tick labels
fig.update_xaxes(tickfont=dict(size=12))
fig.update_yaxes(tickfont=dict(size=12))

# Displaying the figure
fig.show()

By computing the Katz centrality of characters in the series, we can identify which characters play the most important roles in the story and how their influence evolves over time.

Comparing the Katz centrality of different characters can help us understand power dynamics within a story and how characters gain or lose influence over the course of the narrative.

## **Pearson Correlation:** Correlation between different measures

In the fifth book, *Stannis*, *Jon Snow*, and *Daenerys* are the most important characters according to **PageRank**, while *Eddard Stark* follows a similar curve for **degree centrality** and **betweenness centrality**, being important in the first book but fading into obscurity over the series.

We have seen three different measures to calculate the importance of a node in a network: **PageRank**, **betweenness centrality**, and **degree centrality**. All three measures tell us something about the characters and their importance in the co-occurrence network, and we notice some names appearing in all three measures, suggesting a potential correlation between them.

To investigate this, we can calculate the **Pearson correlation** between **PageRank**, **betweenness centrality**, and **degree centrality** for the fifth book. **Pearson correlation** measures the linear correlation between two variables and returns a value between -1 and 1, where -1 indicates a perfectly negative correlation, 0 indicates no correlation, and 1 indicates a perfectly positive correlation.

In [53]:
# Creating a list of pagerank, betweenness centrality, degree centrality
# of all the characters in the fifth book.
measures = [nx.pagerank(books[4]), 
            nx.betweenness_centrality(books[4], weight='weight'), 
            nx.degree_centrality(books[4])]

# Creating the correlation DataFrame
cor = pd.DataFrame.from_records(measures)


import plotly.graph_objs as go

# Creating a list of pagerank, betweenness centrality, degree centrality of all the characters in the fifth book.
measures = [nx.pagerank(books[4]), 
            nx.betweenness_centrality(books[4], weight='weight'), 
            nx.degree_centrality(books[4])]

# Creating the correlation DataFrame
cor = pd.DataFrame.from_records(measures)

# Calculating the correlation
corr_matrix = cor.T.corr()

# Creating a heatmap of the correlation
fig = go.Figure(data=go.Heatmap(z=corr_matrix.values,
                                 x=corr_matrix.columns,
                                 y=corr_matrix.index,
                                 colorscale='RdBu'))

# Updating the layout
fig.update_layout(title="Correlation Between PageRank, Betweenness Centrality, and Degree Centrality for the Fifth Book",
                  xaxis_title="Centrality Measure",
                  yaxis_title="Centrality Measure",
                  width=500, height=500,
                  template='plotly_dark',
                  paper_bgcolor='rgba(0,0,0,0)')

# Updating the font size of the tick labels
fig.update_xaxes(tickfont=dict(size=12))
fig.update_yaxes(tickfont=dict(size=12))

# Displaying the figure
fig.show()

cor.T.corr()

,0,1,2
0,1.000000,0.793372,0.971493
1,0.793372,1.000000,0.833816
2,0.971493,0.833816,1.000000


## **Conclusion**

We have found a high correlation between PageRank, betweenness centrality, and degree centrality for the character co-occurrence network in the fifth book of Game of Thrones.

Throughout our analysis, we have explored different ways to identify the important characters in the co-occurrence network. According to degree centrality, *Eddard Stark* is initially the most important character in the books. However, we have also seen that the importance of characters can change over time and across different centrality measures.

To identify the most important character(s) in the fifth book according to these three measures, we would need to look at the corresponding centrality scores and compare them. Based on the information we have gathered so far, we know that *Stannis*, *Jon Snow*, and *Daenerys* are the most important characters according to PageRank in the fifth book. However, we would need to calculate the betweenness centrality and degree centrality scores for all characters in the fifth book to make a comprehensive comparison across all three measures.

In [54]:
import pandas as pd
import networkx as nx
import plotly.graph_objects as go

# Load the CSV files into a list of DataFrames
books = []
for i in range(1, 6):
    filename = 'datasets/book' + str(i) + '.csv'
    df = pd.read_csv(filename)
    books.append(df)

# Convert DataFrames to MultiGraph objects
graphs = []
for book in books:
    graph = nx.from_pandas_edgelist(book, source='Source', target='Target', edge_attr='weight', create_using=nx.MultiGraph)
    graphs.append(graph)

# Calculate centrality measures for each book
degree_centrality = [nx.degree_centrality(graph) for graph in graphs]
betweenness_centrality = [nx.betweenness_centrality(graph, weight='weight') for graph in graphs]
pagerank = [nx.pagerank(graph) for graph in graphs]

# Get the most important character for each centrality measure in each book
most_important_characters = []
for i in range(len(books)):
    degree_centralities = degree_centrality[i]
    betweenness_centralities = betweenness_centrality[i]
    pagerank_scores = pagerank[i]

    max_degree = max(degree_centralities, key=degree_centralities.get)
    max_betweenness = max(betweenness_centralities, key=betweenness_centralities.get)
    max_pagerank = max(pagerank_scores, key=pagerank_scores.get)

    most_important_characters.append((max_degree, max_betweenness, max_pagerank))

# Create lists for x and y values
x = [f"Book {i+1}" for i in range(len(books))]
degree_values = [most_important_characters[i][0] for i in range(len(books))]
betweenness_values = [most_important_characters[i][1] for i in range(len(books))]
pagerank_values = [most_important_characters[i][2] for i in range(len(books))]

# Create bar traces
trace1 = go.Bar(x=x, y=degree_values, name='Degree Centrality', marker=dict(color='#1a1a40'))
trace2 = go.Bar(x=x, y=betweenness_values, name='Betweenness Centrality', marker=dict(color='#4c0027'))
trace3 = go.Bar(x=x, y=pagerank_values, name='PageRank', marker=dict(color='#1e5128'))

# Create the layout for the bar chart
layout = go.Layout(title='Most Important Characters in Each Book',
                   xaxis_title='Book',
                   yaxis_title='Character',
                   barmode='group',
                   template='plotly_dark',
                   height = 400)

# Create the bar chart
fig1 = go.Figure(data=[trace1, trace2, trace3], layout=layout)

# Create the table data
table_data = []
for i in range(len(books)):
    book_number = i + 1
    table_data.append([f"Book {book_number}", most_important_characters[i][0], most_important_characters[i][1], most_important_characters[i][2]])

# Create the table trace
table_trace = go.Table(header=dict(values=['Book', 'Degree Centrality', 'Betweenness Centrality', 'PageRank']),
                       cells=dict(values=list(zip(*table_data)),
                              fill_color=['#6A359C', '#804FB3','#6A359C', '#804FB3','#6A359C'],  # Set different colors for each row
                        font=dict(color='white')),
                       columnwidth=[0.5, 0.3, 0.4, 0.3],
)



# Create the layout for the table
table_layout = go.Layout(title='Most Important Characters Table',
                         template='plotly_dark',
                         height=400)

# Create the table chart
fig2 = go.Figure(data=[table_trace], layout=table_layout)

# Show the figures
fig1.show()
fig2.show()

Based on the information we have gathered so far, we know that Jon Snow and Daenerys are the most important characters according to PageRank in the books with high centrality scores.



---



# **Thank You**

Github: [Aditya Narendra (Ozzey)](https://github.com/Ozzey/GOT_Network_Analysis))

Copyright: Aditya Narendra